<a href="https://colab.research.google.com/github/Kush-070/Data-Dreamers-5201/blob/main/Copy_of_gpt2_text_generation_with_kerasnlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT2 Text Generation with KerasNLP

**Author:** Chen Qian<br>
**Date created:** 2023/04/17<br>
**Last modified:** 2024/04/12<br>
**Description:** Use KerasNLP GPT2 model and `samplers` to do text generation.

In this tutorial, you will learn to use [KerasNLP](https://keras.io/keras_nlp/) to load a
pre-trained Large Language Model (LLM) - [GPT-2 model](https://openai.com/research/better-language-models)
(originally invented by OpenAI), finetune it to a specific text style, and
generate text based on users' input (also known as prompt). You will also learn
how GPT2 adapts quickly to non-English languages, such as Chinese.

##  Before we begin

Colab offers different kinds of runtimes. Make sure to go to **Runtime ->
Change runtime type** and choose the GPU Hardware Accelerator runtime
(which should have >12G host RAM and ~15G GPU RAM) since you will finetune the
GPT-2 model. Running this tutorial on CPU runtime will take hours.

## Install KerasNLP, Choose Backend and Import Dependencies

This examples uses [Keras 3](https://keras.io/keras_3/) to work in any of
`"tensorflow"`, `"jax"` or `"torch"`. Support for Keras 3 is baked into
KerasNLP, simply change the `"KERAS_BACKEND"` environment variable to select
the backend of your choice. We select the JAX backend below.

In [1]:
!pip install git+https://github.com/keras-team/keras-nlp.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 84.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 92.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 101.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 29.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.16.1 which is incompatible.


In [2]:
import os

os.environ["KERAS_BACKEND"] = "jax"  # or "tensorflow" or "torch"

import keras_nlp
import keras
import tensorflow as tf
import time

keras.mixed_precision.set_global_policy("mixed_float16")

## Introduction to Generative Large Language Models (LLMs)

Large language models (LLMs) are a type of machine learning models that are
trained on a large corpus of text data to generate outputs for various natural
language processing (NLP) tasks, such as text generation, question answering,
and machine translation.

Generative LLMs are typically based on deep learning neural networks, such as
the [Transformer architecture](https://arxiv.org/abs/1706.03762) invented by
Google researchers in 2017, and are trained on massive amounts of text data,
often involving billions of words. These models, such as Google [LaMDA](https://blog.google/technology/ai/lamda/)
and [PaLM](https://ai.googleblog.com/2022/04/pathways-language-model-palm-scaling-to.html),
are trained with a large dataset from various data sources which allows them to
generate output for many tasks. The core of Generative LLMs is predicting the
next word in a sentence, often referred as **Causal LM Pretraining**. In this
way LLMs can generate coherent text based on user prompts. For a more
pedagogical discussion on language models, you can refer to the
[Stanford CS324 LLM class](https://stanford-cs324.github.io/winter2022/lectures/introduction/).

## Introduction to KerasNLP

Large Language Models are complex to build and expensive to train from scratch.
Luckily there are pretrained LLMs available for use right away. [KerasNLP](https://keras.io/keras_nlp/)
provides a large number of pre-trained checkpoints that allow you to experiment
with SOTA models without needing to train them yourself.

KerasNLP is a natural language processing library that supports users through
their entire development cycle. KerasNLP offers both pretrained models and
modularized building blocks, so developers could easily reuse pretrained models
or stack their own LLM.

In a nutshell, for generative LLM, KerasNLP offers:

- Pretrained models with `generate()` method, e.g.,
    `keras_nlp.models.GPT2CausalLM` and `keras_nlp.models.OPTCausalLM`.
- Sampler class that implements generation algorithms such as Top-K, Beam and
    contrastive search. These samplers can be used to generate text with
    custom models.

## Load a pre-trained GPT-2 model and generate some text

KerasNLP provides a number of pre-trained models, such as [Google
Bert](https://ai.googleblog.com/2018/11/open-sourcing-bert-state-of-art-pre.html)
and [GPT-2](https://openai.com/research/better-language-models). You can see
the list of models available in the [KerasNLP repository](https://github.com/keras-team/keras-nlp/tree/master/keras_nlp/models).

It's very easy to load the GPT-2 model as you can see below:

In [3]:
# To speed up training and generation, we use preprocessor of length 128
# instead of full length 1024.
preprocessor = keras_nlp.models.GPT2CausalLMPreprocessor.from_preset(
    "gpt2_base_en",
    sequence_length=128,
)
gpt2_lm = keras_nlp.models.GPT2CausalLM.from_preset(
    "gpt2_base_en", preprocessor=preprocessor
)

100%|██████████| 448/448 [00:00<00:00, 245kB/s]
100%|██████████| 0.99M/0.99M [00:01<00:00, 802kB/s]
100%|██████████| 446k/446k [00:01<00:00, 450kB/s]
100%|██████████| 484/484 [00:00<00:00, 333kB/s]
100%|██████████| 475M/475M [00:31<00:00, 15.7MB/s]


Once the model is loaded, you can use it to generate some text right away. Run
the cells below to give it a try. It's as simple as calling a single function
*generate()*:

In [4]:
start = time.time()

output = gpt2_lm.generate("My trip to Yosemite was", max_length=200)
print("\nGPT-2 output:")
print(output)

end = time.time()
print(f"TOTAL TIME ELAPSED: {end - start:.2f}s")


GPT-2 output:
My trip to Yosemite was a great success and a great adventure, I was able to get to the summit of the mountain in a matter of days.
I went with my friends and my dog to the peak, which is a great spot to get to know a little bit about the area.
We went to the summit with the help of our guide, who gave us an overview about the surrounding mountains, as well as the views.
The next day, we headed down to the top to see the beautiful scenery of the Yosemite Valley.
The next day we went down to the summit and saw a beautiful view of the Yosemite Valley and the surrounding Yosemite Valley. I was so excited that I decided to hike the route down to see the beautiful Yosemite Valley.
I was very excited about the views, and I really like seeing the view from the summit. I was so proud of it, I decided to hike the route down and saw the beautiful view.
After the route down to
TOTAL TIME ELAPSED: 9.35s


Try another one:

In [5]:
start = time.time()

output = gpt2_lm.generate("That Italian restaurant is", max_length=200)
print("\nGPT-2 output:")
print(output)

end = time.time()
print(f"TOTAL TIME ELAPSED: {end - start:.2f}s")


GPT-2 output:
That Italian restaurant is the first Italian restaurant to be named "The Best Italian Restaurant in the World" by TripAdvisor. The restaurant is named after Italian restaurant owner and owner, Luigi Sacco, who was the owner of the Italian restaurant in Milan for over 30 years.

Sacco is the founder and owner of Sacco's Restaurant and Restaurant Group (SRG) and has been a member of the Sacco family since 1997. He has also served as the CEO and President of the company for the past 15 years. Sacco is also the founder and chairman of the Italian restaurant company, Sacco's Restaurant Group.

Sacco's restaurant is located at the intersection of the Via di Giorgio and Via della Roma. The restaurant features a number of different dishes, from Italian and Italian-American cuisine, to Italian-American cuisine from around the world.

Sacco's Restaurant Group is owned by his son, Luigi Sacco, Sr. and is
TOTAL TIME ELAPSED: 2.43s


Notice how much faster the second call is. This is because the computational
graph is [XLA compiled](https://www.tensorflow.org/xla) in the 1st run and
re-used in the 2nd behind the scenes.

The quality of the generated text looks OK, but we can improve it via
fine-tuning.

## More on the GPT-2 model from KerasNLP

Next up, we will actually fine-tune the model to update its parameters, but
before we do, let's take a look at the full set of tools we have to for working
with for GPT2.

The code of GPT2 can be found
[here](https://github.com/keras-team/keras-nlp/blob/master/keras_nlp/models/gpt2/).
Conceptually the `GPT2CausalLM` can be hierarchically broken down into several
modules in KerasNLP, all of which have a *from_preset()* function that loads a
pretrained model:

- `keras_nlp.models.GPT2Tokenizer`: The tokenizer used by GPT2 model, which is a
    [byte-pair encoder](https://huggingface.co/course/chapter6/5?fw=pt).
- `keras_nlp.models.GPT2CausalLMPreprocessor`: the preprocessor used by GPT2
    causal LM training. It does the tokenization along with other preprocessing
    works such as creating the label and appending the end token.
- `keras_nlp.models.GPT2Backbone`: the GPT2 model, which is a stack of
    `keras_nlp.layers.TransformerDecoder`. This is usually just referred as
    `GPT2`.
- `keras_nlp.models.GPT2CausalLM`: wraps `GPT2Backbone`, it multiplies the
    output of `GPT2Backbone` by embedding matrix to generate logits over
    vocab tokens.

## Finetune on Reddit dataset

Now you have the knowledge of the GPT-2 model from KerasNLP, you can take one
step further to finetune the model so that it generates text in a specific
style, short or long, strict or casual. In this tutorial, we will use reddit
dataset for example.

In [6]:
import tensorflow_datasets as tfds
imdb_reviews_ds = tfds.load("imdb_reviews", split="train", as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.EFEPAW_1.0.0/imdb_reviews-train.tfrecor…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.EFEPAW_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.EFEPAW_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [7]:
for document, title in imdb_reviews_ds:
    print(document.numpy())
    print(title.numpy())
    break

b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
0


Let's take a look inside sample data from the reddit TensorFlow Dataset. There
are two features:

- **__document__**: text of the post.
- **__title__**: the title.

In [8]:
train_ds = (
    imdb_reviews_ds.map(lambda document, _: document)
    .batch(32)
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)

In [9]:
train_ds = train_ds.take(500)
num_epochs = 1

# Linearly decaying learning rate.
learning_rate = keras.optimizers.schedules.PolynomialDecay(
    5e-5,
    decay_steps=train_ds.cardinality() * num_epochs,
    end_learning_rate=0.0,
)
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
gpt2_lm.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=loss,
    weighted_metrics=["accuracy"],
)

gpt2_lm.fit(train_ds, epochs=num_epochs)

500/500 ━━━━━━━━━━━━━━━━━━━━ 204s 341ms/step - accuracy: 0.3225 - loss: 3.6343


In [10]:
start = time.time()

output = gpt2_lm.generate("I like basketball", max_length=200)
print("\nGPT-2 output:")
print(output)

end = time.time()
print(f"TOTAL TIME ELAPSED: {end - start:.2f}s")


GPT-2 output:
I like basketball. It's one of those things that I like so much that I can't wait to watch it. I can't wait to see it again! I'm a big fan of the NBA, but it was the first time I watched a film with a cast that I could actually watch. That was a great experience. I'm glad that the film is on DVD because it shows how the NBA has evolved over the last few years and the players are still playing in the same league. I can only hope that the film is released on a DVD and
TOTAL TIME ELAPSED: 6.84s


## Into the Sampling Method

In KerasNLP, we offer a few sampling methods, e.g., contrastive search,
Top-K and beam sampling. By default, our `GPT2CausalLM` uses Top-k search, but
you can choose your own sampling method.

Much like optimizer and activations, there are two ways to specify your custom
sampler:

- Use a string identifier, such as "greedy", you are using the default
configuration via this way.
- Pass a `keras_nlp.samplers.Sampler` instance, you can use custom configuration
via this way.

In [11]:
# Use a string identifier.
gpt2_lm.compile(sampler="top_k")
output = gpt2_lm.generate("I like basketball", max_length=200)
print("\nGPT-2 output:")
print(output)

# Use a `Sampler` instance. `GreedySampler` tends to repeat itself,
greedy_sampler = keras_nlp.samplers.GreedySampler()
gpt2_lm.compile(sampler=greedy_sampler)

output = gpt2_lm.generate("I like basketball", max_length=200)
print("\nGPT-2 output:")
print(output)


GPT-2 output:
I like basketball, and this movie has the potential to go beyond that. The characters, the plot, the story, and the acting are all very well crafted. The movie is a good one for the kids who like basketball, but they can't get over the fact that they're all in it together and that it's just not a fun game. I'm not sure why this movie was made, and I'm not sure why they did this, because they were just too busy to make a movie. The acting, the story, and the acting are all very well crafted, and the movie has the potential to go beyond that. The movie is a good one for the kids who like basketball, and they can't get over the fact that

GPT-2 output:
I like basketball, but I don't like the movie. I don't like the movie. I don't like the movie. I don't like the movie. I don't like the movie. I don't like the movie. I don't like the movie. I don't like the movie. I don't like the movie. I don't like the movie. I don't like the movie. I don't like the movie. I don't like the

For more details on KerasNLP `Sampler` class, you can check the code
[here](https://github.com/keras-team/keras-nlp/tree/master/keras_nlp/samplers).

## Finetune on Chinese Poem Dataset

We can also finetune GPT2 on non-English datasets. For readers knowing Chinese,
this part illustrates how to fine-tune GPT2 on Chinese poem dataset to teach our
model to become a poet!

Because GPT2 uses byte-pair encoder, and the original pretraining dataset
contains some Chinese characters, we can use the original vocab to finetune on
Chinese dataset.

In [12]:
!# Load chinese poetry dataset.
!git clone https://github.com/chinese-poetry/chinese-poetry.git

Cloning into 'chinese-poetry'...
remote: Enumerating objects: 7323, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 7323 (delta 5), reused 10 (delta 3), pack-reused 7309
Receiving objects: 100% (7323/7323), 236.98 MiB | 13.39 MiB/s, done.
Resolving deltas: 100% (5003/5003), done.
Updating files: 100% (2285/2285), done.


Load text from the json file. We only use《全唐诗》for demo purposes.

In [13]:
import os
import json

poem_collection = []
for file in os.listdir("chinese-poetry/全唐诗"):
    if ".json" not in file or "poet" not in file:
        continue
    full_filename = "%s/%s" % ("chinese-poetry/全唐诗", file)
    with open(full_filename, "r") as f:
        content = json.load(f)
        poem_collection.extend(content)

paragraphs = ["".join(data["paragraphs"]) for data in poem_collection]

Let's take a look at sample data.

In [14]:
print(paragraphs[0])

增添庾郎方丈，掇拾儀休棄遺。但令門人學圃，不妨夫子下帷。


Similar as Reddit example, we convert to TF dataset, and only use partial data
to train.

In [15]:
train_ds = (
    tf.data.Dataset.from_tensor_slices(paragraphs)
    .batch(16)
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)

# Running through the whole dataset takes long, only take `500` and run 1
# epochs for demo purposes.
train_ds = train_ds.take(500)
num_epochs = 1

learning_rate = keras.optimizers.schedules.PolynomialDecay(
    5e-4,
    decay_steps=train_ds.cardinality() * num_epochs,
    end_learning_rate=0.0,
)
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
gpt2_lm.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=loss,
    weighted_metrics=["accuracy"],
)

gpt2_lm.fit(train_ds, epochs=num_epochs)

500/500 ━━━━━━━━━━━━━━━━━━━━ 114s 177ms/step - accuracy: 0.2535 - loss: 2.6651


Let's check the result!

In [16]:
output = gpt2_lm.generate("昨夜雨疏风骤", max_length=200)
print(output)

昨夜雨疏风骤，曉疏霽爲風。清綠馳露，清書溪花。聞香苦露，馬霜風驚。清清知風沉，淺池風聞。風草


In [17]:
import json # Import the json module

with open('/content/titanic.json', 'r', encoding='utf-8') as f:
    titanic_data = json.load(f) # Now you can use json.load()

Not bad 😀

In [18]:
# Create a TensorFlow dataset from the JSON data
texts = titanic_data["text"]
titanic_ds = tf.data.Dataset.from_tensor_slices(texts)

In [19]:
# Prepare the dataset for training
# Prepare the dataset for training
train_ds = (
    titanic_ds
    .batch(32)
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)

In [20]:
# Take a subset for training (optional, depending on your needs)
train_ds = train_ds.take(1000)
num_epochs = 1

# Configure a dynamically decreasing learning rate based on linear decay
learning_rate = keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=5e-5,
    decay_steps=train_ds.cardinality() * num_epochs,
    end_learning_rate=0.0,
)

In [21]:
# Define the loss function
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Assuming gpt2_lm is your GPT-2 model
# If not already defined, you need to load or create your GPT-2 model here

# Model compilation
gpt2_lm.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=loss,
    weighted_metrics=["accuracy"],
)

# Train the model
gpt2_lm.fit(train_ds, epochs=num_epochs)

2/2 ━━━━━━━━━━━━━━━━━━━━ 55s 36s/step - accuracy: 0.0056 - loss: 1.4846


In [22]:
# Inference
start = time.time()
output = gpt2_lm.generate("I'm the king of the world", max_length=200)
print("\nGPT-2 output:")
print(output)
end = time.time()
print(f"TOTAL TIME ELAPSED: {end - start:.2f}s")


GPT-2 output:
I'm the king of the world落起臺，霜霜花清河。聞聞聚臺青，苦風爲樹疎。曲塞山坐爾，靜自更處處。自爭爭處處，爲處
TOTAL TIME ELAPSED: 6.86s


#Real Time Dataset

In [24]:
import json
with open('/content/realtime_dataset.json', 'r', encoding='utf-8') as f:realtime_data=json.load(f)

In [25]:
# Extract texts from the dataset
texts = [realtime_data.get('text', '')]  # Access the 'text' value directly, or provide an empty string if it doesn't exist

# Create a TensorFlow dataset from the texts
improv_ds = tf.data.Dataset.from_tensor_slices(texts)

# Prepare the dataset for training
train_ds = (
    improv_ds
    .batch(32)
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)

In [26]:
# Take a subset for training (optional, depending on your needs)
train_ds = train_ds.take(1000)
num_epochs = 3

# Configure a dynamically decreasing learning rate based on linear decay
learning_rate = keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=5e-5,
    decay_steps=train_ds.cardinality() * num_epochs,
    end_learning_rate=0.0,
)

# Define the loss function
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Model compilation
gpt2_lm.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=loss,
    weighted_metrics=["accuracy"],
)

# Train the model
gpt2_lm.fit(train_ds, epochs=num_epochs)

Epoch 1/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 22s 22s/step - accuracy: 0.0234 - loss: 8.3352
Epoch 2/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 17s 17s/step - accuracy: 0.0156 - loss: 8.4033
Epoch 3/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.0391 - loss: 6.7527


In [27]:
# Inference
start = time.time()
output = gpt2_lm.generate("kushal is", max_length=200)
print("\nGPT-2 output:")
print(output)
end = time.time()
print(f"TOTAL TIME ELAPSED: {end-start:.2f}s")


GPT-2 output:
kushal is苔發發，青至清青。秋山書清清，山游陰風。苦羣陵堂，積積空苦樂。秋風秋秋霧，露露露清。清
TOTAL TIME ELAPSED: 6.13s
